# Сравнение open-source солверов на примере задачи ритейла

Привет, Habr! На связи отдел аналитики данных X5 Tech.

В этой статье мы продолжим говорить про оптимизаторы.
Для каких задач и как они могут применяться в python мы поговорили в [предыдущей статье](http://), также там были описаны основные типы оптимизационных задач, которые можно встретить на практике.

Здесь мы сделаем краткий обзор существующих open-source решений в python, затронем их различия, особенности и задачи, которые можно решать с их помощью.


## Обзор пакетов python


Рассмотрим open-source пакеты и солверы для решения задач условной оптимизации.

### Scipy 

[__Scipy__](https://scipy.org/) - одна из первых библиотек в python, знакомство с которой начинается у специалистов в области Data(Computer) Science - она содержит большой набор функций для научных вычислений,
в том числе имеет инструменты для решения оптимизационных задач, находящиеся в модуле __scipy.optimize__.
В этом модуле находятся методы для решения задач как нелинейного программирования (__NLP__) так и линейного программирования (__LP__), в том числе задач смешанного целочисленного линейного програмимрования(__MILP__).


Среди солверов, которые поддерживают решение задач условной оптимизации (NLP) можно выделить ___cobyla___, ___slsqp___, ___trust-constr___. Описание методов, ссылки на статьи и примеры применения солверов можно найти [тут](https://habr.com/ru/company/ods/blog/448054/). Здесь же вкратце отметим, что __сobyla__ - это метод, позволяющий производить оптимизацию функции, градиент которой неизвестен, т.е. по сути заниматься оптимизацией "черного ящика". Также стоит обратить внимание на то, что cobyla не поддерживает ограничения типа равенства и границы для переменных $x$, задаваемых через параметр bounds в функции __minimize__(из модуля scipy.optimi
ze) - их необходимо задавать через линейные ограничения, например с помощью LinearConstraint. Что каксается __slsqp__, __trust-constr__, то это методы уже второго порядка. Для увеличения устойчивости и сходимости оптимизаторов, функции(целевую и ограничения) необходимо отмасштабировать так, чтобы принимаемые значения имели порядок единицы, в противном случае решение может "застрять" либо в начальной точке, либо просто не дойти до локального оптимума

Рассмотрим небольшой пример применения метода minimize, илюстрирующий полезность масштабирования, основанный на постановке задачи ценообразования из [статьи 1](link) - максимизация выручки с сохранением текущего уровня маржи с ограниченным диапазоном изменения цены в пределах ±10% от текущей. Обозначения:

$n$ - количество товаров

$P_{0, i}$ - текущая цена,

$Q_{0, i}$ - текущие продажи,

$x_{i} = P_{i} / P_{0, i}$ - отношение новой цены $P_{i}$ к текущей

$E_{i}$ - параметр для пересчета спроса по формуле $Q_{i}(x) = Q_{0, i} \cdot \exp(E \cdot (x - 1))$

$R_{i}(x_{i}) = \sum_{i=1}^{n} P_{0, i} \cdot x_{i} \cdot Q_{i}(x) $ - выручка

$M_{i}(x_{i}) = \sum_{i=1}^{n} (P_{0, i} \cdot x_{i} - C_{i})\cdot Q_{0, i} \cdot Q_{i}(x_i) $ - маржа,

$M_{0} = \sum_{i=1}^{n} M_{i}(x_{i}=1)$ - текущая маржа,

\begin{cases}
\tag{1}
\sum_{i=1}^{n} R_{i}(x_{i}) \to \max,
\\
\sum_{i=1}^{n} M_{i}(x_{i}) \geqslant M_{0},
\\
x_i \in [0.9, 1.1], \ i=1..n\\
\end{cases}

In [29]:
# мини пример из постановки первой статьи
import scipy.optimize as scopt
import numpy as np

# задаем параметры E, используемые в формуле Q = Q0 * exp(E * (x - 1))
E = np.array([-3., -1., -0.5])
# текущие цены
P0 = np.array([10., 10., 10.])
# текущие продажи
Q0 = np.array([500000., 2000000., 300000.0])
# себестоимость
C = np.array([9.0, 8.0, 7.0])
# текущая выручка
R0 = np.sum(P0 * Q0)
# текущая маржа
M0 = np.sum((P0 - C) * Q0)

# выручка - целевая функция, задаем возможность "управлять" масштабом через 'scale'
def f_obj(x, args):
    f = - args['scale'] * np.sum(Q0 * P0 * x * np.exp(E * (x-1.)))
    return f
obj = f_obj

# функция для ограничения по марже, по умолчанию отмасштабиируем ограничения на текущую выручку
def f_cons(x):
    f = np.sum(Q0 * (P0 * x - C) * np.exp(E * (x-1.0))) / R0
    return f
cons = [scopt.NonlinearConstraint(f_cons, lb=M0 / R0, ub=np.inf)]

# поиск новой цены производим в диапазоне 90% - 110% от текущей цены
x_bounds = [(0.9, 1.1)] * 3
# стартовая точка для поиска
x0 = [1.0] * 3

res_nonscaled = scopt.minimize(obj, x0, bounds=x_bounds, constraints=cons, method='slsqp', args={'scale': 1.})
res_scaled = scopt.minimize(obj, x0, bounds=x_bounds, constraints=cons, method='slsqp', args={'scale': 1.0 / R0})
print('Решение с масштабированием ', np.round(res_scaled['x'], 3), res_scaled['message'])
print('Значение функции ', round(f_obj(res_scaled['x'], args={'scale': 1.0}), ))
print('Значенте маржи', round(R0 * cons[0].fun(res_scaled['x']), 1), ' и M0', M0)
print('Решение без масштабирования',np.round(res_nonscaled['x'], 3), res_nonscaled['message'])


Решение с масштабированием  [0.9   1.016 1.1  ] Optimization terminated successfully
Значение функции  -29210742
Значенте маржи 5399999.4  и M0 5400000.0
Решение без масштабирования [1. 1. 1.] Optimization terminated successfully


Как мы видим, в случае когда целевая функция не отмасштабирована солвер даже не трогается с места - решение это стартовое значение, более того согласно своим внутренним критериям считает, что решение является оптимальным, хотя на деле это не так - этот момент необходимо иметь это ввиду при реализации.

Для решения задач линейного программирования в подмодуле __optimize__ имеется функция [__linprog__](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html#scipy.optimize.linprog), начиная с версии scipy==1.9.0 появилась возможность решения задачи линейного целочисленного программирования с помощью функции [__milp__](https://scipy.github.io/devdocs/reference/generated/scipy.optimize.milp.html) и __linprog__. В качестве солвера LP(MILP) по умолчанию использется [HiGHS](https://www.maths.ed.ac.uk/hall/HiGHS/) - в нем реализован симплекс метод(highs-ds) и метод внутренней точки(highs-ipm) - солвер автоматически выбирает один из методов, но при запуске можно самоятоятеьно выбрать один из них. 


Для примера рассмотрим небольшую задачку целочисленного рюкзака, где для наглядности применения метода __linprog__ используем переменные разных типов:

\begin{cases}
\tag{2}
x = (x_{1}, x_{2}, x_{3}, x_{4})
\\
\\
F(x) = 1 \cdot x_{1} + 2 \cdot x_{2} + 3 \cdot x_{3} + 1 \cdot x_{4} \to \max
\\
\\
2 \cdot x_1 + 1 \cdot x_2 + 3 \cdot x_3 + 1 \cdot x_4 \leqslant 7.5
\\
\\
x_{1}, x_{3} \in \{0, 1\}, x_{2} \in \{0, 1, 2\}, x \in [0.0; 0.5]
\end{cases}

In [10]:
# пример с "рюкзаком" у которого типы переменных различаются
# так как задача максимизации, не забываем ставить минус
c = -np.array([1., 2., 3., 1.])
A_ub = np.array([[2., 1., 3., 1.]])
b_ub = np.array([7.5])
# проставляем индикаторы для типа переменных, 0 - непрерывное, 1 - целое число
var_types = [1, 1, 1, 0]
# также указываем границы, в том числе и для целочисленных переменных
bounds = [(0, 1), (0, 2), (0, 1), (0, 1)]
res_milp = scopt.linprog(c=c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, integrality=var_types, method='highs')
res_milp

           con: array([], dtype=float64)
 crossover_nit: -1
         eqlin:  marginals: array([], dtype=float64)
  residual: array([], dtype=float64)
           fun: -8.5
       ineqlin:  marginals: array([0.])
  residual: array([0.])
         lower:  marginals: array([0., 0., 0., 0.])
  residual: array([1. , 2. , 1. , 0.5])
       message: 'Optimization terminated successfully. (HiGHS Status 7: Optimal)'
           nit: -1
         slack: array([0.])
        status: 0
       success: True
         upper:  marginals: array([0., 0., 0., 0.])
  residual: array([0. , 0. , 0. , 0.5])
             x: array([1. , 2. , 1. , 0.5])

Как мы видим задача успешно решена, относительно несложным перебором, благо вариантов не так много, можно убедиться в том что это действительно оптимальное решение.

### Pyomo

[__Pyomo__](http://www.pyomo.org/) - пакет, который содержит ряд инструментов для формулирования, решения и анализа оптимизационных моделей.
Главная особенность — это удобный интерфейс для структурированного формулирования оптимизационной задачи и поддержка большого количества солверов, в том числе коммерческих.
Pyomo внутри себя преобразует сформулированную модель в формат, понятный для запускаемого солвера.

Pyomo входит в проект [COIN-OR](https://www.coin-or.org/), содержащий ряд солверов, среди которых выделим два:

[___Ipopt___](https://github.com/coin-or/Ipopt) - находит локальные оптимумы в задаче NLP с помощью прямо-двойственного метода внутренней точки, подробнее в оригинальной [статье](http://www.optimization-online.org/DB_HTML/2004/03/836.html).

[___Cbc___](https://github.com/coin-or/Cbc) - решает задачи MILP, на базе алгоритма, сочетающем в себе метод ветвей и границ и секущих плоскостей [wiki](https://en.wikipedia.org/wiki/Branch_and_cut).

Также для решения задач LP(MILP) имеется поддержка пакета [glpk](https://en.wikipedia.org/wiki/GNU_Linear_Programming_Kit).
Отметим еще один солвер [bonmin](https://github.com/coin-or/Bonmin), который построен поверх __Cbc__ и __Ipopt__ - сочетание двух солверов, который позволяет браться за заадачи __MINLP__.

Процесс построения оптимизационной модели в pyomo состоит и основных этапов: создание объекта оптимизационной модели, объявление переменных в этой модели, формулирование целевой функции, описание ограничений, запуск солвера, решающего задачу, рассмотрим шаги на примере задачи (1) и (2)

In [33]:
# пример аналогичный предыдущему
# мини пример из постановки первой статьи
import pyomo.environ as pyo
import numpy as np

# Количество товаров
N = 3
# задаем эластичности, используемые в формуле Q = Q0 * exp(E * (P/P0 - 1))
E = np.array([-3., -1., -0.5])
# текущие цены
P0 = np.array([10., 10., 10.])
# текущие продажи
Q0 = np.array([500000., 2000000., 300000.0])
# себестоимость
C = np.array([9.0, 8.0, 7.0])
# текущая выручка
R0 = np.sum(P0 * Q0)
# текущая маржа
M0 = np.sum((P0 - C) * Q0)
# диапазон поиска переменных
bounds = [(0.9, 1.1)] * N
# объявление объекта - модели 
model = pyo.ConcreteModel('model')
# задаем переменные, в данном случае они все непрерывные, инициализиируем 1.0
model.x = pyo.Var(range(N), domain=pyo.Reals, bounds=bounds, initialize=1)
# объявление целевой функции и передача в модель
obj_expr = sum(scale * P0[i] * model.x[i] * Q0[i] * pyo.exp(E[i] * (model.x[i] - 1)) for i in model.x)
model.obj = pyo.Objective(expr=obj_expr, sense=pyo.maximize)
# объявление ограничения и передача в модель
con_expr = sum((P0[i] * model.x[i] - C[i]) * Q0[i] * pyo.exp(E[i] * (model.x[i] - 1)) for i in model.x) >= M0
model.con = pyo.Constraint(expr=con_expr)
# запуск солвера ipopt для решения поставленной оптимизационной задачи
solver = pyo.SolverFactory('ipopt')
res = solver.solve(model)
# получение ответа - результата решения задачи
x_opt = [round(model.x[i].value, 3) for i in model.x]
print(x_opt, '; obj value = ', round(model.obj(x_opt), 0), round(model.con(x_opt), 1))

[0.9, 1.016, 1.1] ; obj value =  29210742.0 5400000.0


In [39]:
# milp примерчик с рюкзаком
# пример с "рюкзаком" у которого типы переменных различаются
c = np.array([1., 2., 3.])
A = np.array([2., 1., 3.])
b = 7.5

model = pyo.ConcreteModel('model')
bounds = [(0, 1), (0, 2), (0, 1)]

model.x = pyo.Var(range(3), domain=pyo.Integers, bounds=bounds)

obj_expr = sum(c * model.x)
model.obj = pyo.Objective(expr=obj_expr, sense=pyo.maximize)

con_expr = sum(A * model.x) <= b
model.con = pyo.Constraint(expr=con_expr)

solver = pyo.SolverFactory('glpk')
res = solver.solve(model)
x_opt = [model.x[i].value for i in model.x]
print(x_opt, '; obj value = ', model.obj(x_opt))

[1.0, 2.0, 1.0] ; obj value =  8.0


Pyomo имеет подмодуль __GDP__ (Generalized Disjunctive Programming) - который позволяет моделировать логические правила и задавать ограничения, которые должны при этом выполняться, например, в простейшем случае данный подход может быть применен когда необходимо выбрать одно из действий, каждое из которых описывается своей системой ограничений.

Разберем чем может быть полезен данный подход на примере следующей задачи:

\begin{cases}
\tag{3}
x = (x_{1}, x_{2}, x_{3})
\\
\\
F(x) = (x - 0)^2 + (x - 1)^2 + (x + 2)^2
\\
\\
x_i \in [-3; -1.5 ] \cup \{0\} \cup [ 1.5; 3] , \ i = 1..3
\end{cases}

Как несложно заметить, здесь в опрелении области переменных есть "разрыв", который тривиальным образом описать невозможно, необходимо вводить условие на то в какой из трех областей $[-3; -1.5 ], \{0\}, [ 1.5; 3]$ необходимо производить поиск решение.


In [5]:
import pyomo.environ as pyo
from pyomo.gdp import Disjunct, Disjunction
model = pyo.ConcreteModel('gdp_sample')
model.x = pyo.Var(range(0, 3), domain=pyo.Reals, bounds=(-3., 3.))
a = [0.0, 1.0, -2.0]
obj_expr = sum((model.x[i] - a[i]) ** 2 for i in model.x)
model.obj = pyo.Objective(expr=obj_expr, sense=pyo.minimize)
model.djn = Disjunction(range(3))

d = Disjunct()
d.c = pyo.Constraint(rule=(0, model.x[0], 1))
for i in range(3):
    model.djn[i] = [model.x[i] <= -1.5, model.x[i] == 0, model.x[i] >= 1.5] 

    
pyo.TransformationFactory('gdp.hull').apply_to(model)
solver = pyo.SolverFactory('bonmin')
res = solver.solve(model)
x_opt = [round(model.x[i].value, 3) for i in model.x]
print('solution is', x_opt)


solution is [0.0, 1.5, -2.0]


[__Cvxpy__](https://www.cvxpy.org/index.html) - данный пакет был реализован для решения задач [выпуклой оптимизации](convex optimization).
После того как задача сформулирована, перед решением проверяется выпуклость и аффинность целевой функции и ограничений с помощью правил [DCP](https://www.cvxpy.org/tutorial/dcp/index.html)
(disciplined convex programming). По сути это набор правил, которые однозначно гарантируют выпуклость функции. После проверки задача преобразуется в стандартную форму и передается квадратичному или коническому солверу.
Полный список солверов и, соответственно, типов решаемых задач можно найти [здесь](https://www.cvxpy.org/tutorial/advanced/index.html).


In [40]:
import cvxpy as cp

# пример, когда не выполняются правила DCP, но при этом функция выпуклая
X1, Y1 = 0.0, 0.0
X2, Y2 = 1.0, 2.0
# N = V2 / V1
N = 1.5

X = cp.Variable(1)
Y_ = 1.0

objective = cp.norm(cp.hstack([X-X1, Y_-Y1]), 2) + N * cp.norm(cp.hstack([X2-X, Y2-Y_]), 2)
constraints = []
constraints.extend([X >= 0.0, X <= X2])
problem = cp.Problem(cp.Minimize(objective), constraints)
obj_val = problem.solve('ECOS')
X_ = X.value[0]
print(f'x_opt = {round(X_, 3)}')

x_opt = 0.623


In [38]:
objective = cp.sqrt(cp.square(X - X1) + cp.square(Y_ - Y1) ** 2) + cp.sqrt(cp.square(X2 - X) + cp.square(Y2 - Y_) ** 2)
constraints = []
constraints.extend([X >= 0.0, X <= X2])
problem = cp.Problem(cp.Minimize(objective), constraints)
problem.is_dcp(), problem.is_dgp(), problem.is_dpp(), problem.is_dqcp(), problem.is_qp()

(False, False, False, False, False)

In [114]:
x = cp.Variable(3, boolean=True)
c = np.array([1., 2., 3.])
A = np.array([2., 1., 3.])
b = 5.
obj = cp.Maximize(cp.sum(c @ x))
cons = [(A @ x) <= b]
prb = cp.Problem(obj, cons)
sol = prb.solve(verbose=False, solver='GLPK_MI')

Составим таблицу для перечисленных пакетов и солверов, какие типы задач они решают:


|  Пакеты в python  |     Солвер(метод)    | NLP | LP | MILP | MINLP |
|-------------------|----------------------|-----|----|------|-------|
| scipy             | cobyla               | y   | n  | n    | n     |
| scipy             | slsqp                | y   | n  | n    | n     |
| scipy             | trust-constr         | y   | n  | n    | n     |
| scipy             | highs                | n   | y  | y    | n     |
| pyomo             | ipopt                | y   | y  | n    | n     |
| pyomo, cvxpy      | glpk                 | n   | y  | y    | n     |
| pyomo, cvxpy      | cbc                  | n   | y  | y    | n     |
| cvxpy             | ecos                 | y   | y  | y    | n     |
| pyomo             | bonmin               | y   | y  | y    | y     |



## Заключение

В данной статье мы рассмотрели ряд open-source библиотеки для решения оптимизационых задач